<a href="https://colab.research.google.com/github/jborders23/math6397-funhouse/blob/main/MATH6397_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Processing

Import & Pre-Process Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load data from GitHub
tesla = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/TSLA.csv")
ford = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/F.csv")
gm = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/GM.csv")
sp500 = pd.read_excel("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/SP500.xlsx")
#reverse sp500 and reset index
sp500 = sp500.iloc[::-1].reset_index(drop=True)


In [3]:
#keep date and closing price for each security
t = tesla.iloc[:,[0,4,7]] #keep 'tomorrow' price for tesla, the target stock
f = ford.iloc[:,[0,4]]
g = gm.iloc[:,[0,4]]
s = sp500.iloc[:,[0,4]]

#add column to tesla that equals 1 if tomorrow>close
t['Movement'] = np.where(t['Tomorrow'] > t['Close'],1,0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
#concatenate dfs
data = pd.concat(objs=[f['Date'],s['Close'],f['Close'],g['Close'],t['Close'],t['Tomorrow'],t['Movement']],axis=1,keys=['Date','SP','Ford','GM','Tesla','Tomorrow','Movement'])
#normalize the time series: xi - min(xi)/max(xi) - min(xi)
s_data = data #will work with a standardized data frame
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
s_data[['SP','Ford','GM','Tesla','Tomorrow']] = scaler.fit_transform(s_data[['SP','Ford','GM','Tesla','Tomorrow']])
s_data.head()

,Date,SP,Ford,GM,Tesla,Tomorrow,Movement
0,4/26/2016,0.041722,0.966270,0.340939,0.017649,0.017113,0
1,4/27/2016,0.043301,0.957341,0.340275,0.017113,0.016226,0
2,4/28/2016,0.034450,1.000000,0.346478,0.016226,0.014585,0
3,4/29/2016,0.029639,0.947421,0.332300,0.014585,0.014831,1
4,5/2/2016,0.037022,0.953373,0.331192,0.014831,0.012593,0


# Convolution

https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

Prepare Data for Convolution

In [5]:
# split s_data into what the network needs
d_data = s_data[['SP','Ford','GM','Tesla','Movement']]
X = d_data.iloc[:,0:4]
y = d_data.iloc[:,-1]
#convert X to array
X = np.array(X[:])

Train/Test Split: Unsure whether to split randomly
or to do standard time series move of first 80% train, last 20% test...right now it's random, can be changed easily.

In [ ]:
#train/test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [7]:
#reshape to 3d for convolution network
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

764     0
1155    1
831     0
1132    0
582     0
       ..
704     0
597     1
411     1
1195    1
613     0
Name: Movement, Length: 1005, dtype: int64

Building the Model

In [19]:
batch_size = int(round(len(x_train)**0.5,0))
num_classes = 2
epochs = 100
input_shape=(x_train.shape[1], 1)

In [20]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

#build
model = models.Sequential()
model.add(layers.Conv1D(filters=64,kernel_size=10,padding='same',activation='relu',input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64,kernel_size=10,padding='same',activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
#check to see if a normal mlp 'works'
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2,activation='softmax'))
checkpointer = ModelCheckpoint('does mlp work?', monitor='val_accuracy', save_best_only=True)
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
monitor = model.fit(x_train, tf.one_hot(pd.Series(y_train,dtype='category').astype('category').cat.codes,2), epochs=100, batch_size= batch_size,
                    validation_data=(x_test, tf.one_hot(pd.Series(y_test, dtype='category').astype('category').cat.codes,2)), callbacks=[checkpointer])


Epoch 1/100
32/32 [==============================] - 1s 13ms/step - loss: 0.6932 - accuracy: 0.4931 - val_loss: 0.6899 - val_accuracy: 0.5397
INFO:tensorflow:Assets written to: does mlp work?/assets
Epoch 2/100
32/32 [==============================] - 0s 5ms/step - loss: 0.6927 - accuracy: 0.5207 - val_loss: 0.6914 - val_accuracy: 0.5397
Epoch 3/100
32/32 [==============================] - 0s 5ms/step - loss: 0.6923 - accuracy: 0.5302 - val_loss: 0.6920 - val_accuracy: 0.5198
Epoch 4/100
32/32 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.5033 - val_loss: 0.6903 - val_accuracy: 0.5397
Epoch 5/100
32/32 [==============================] - 0s 5ms/step - loss: 0.6925 - accuracy: 0.4962 - val_loss: 0.6908 - val_accuracy: 0.5397
Epoch 6/100
32/32 [==============================] - 0s 6ms/step - loss: 0.6922 - accuracy: 0.5121 - val_loss: 0.6909 - val_accuracy: 0.4722
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.6910 - accuracy: 0.49

pretty crappy performance, but the data is shaped correctly! (probably)...now just need to get flattened layers to a random forest